<a href="https://colab.research.google.com/github/Tianea2160/PythonPractice/blob/master/NetworkConfig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


In [14]:
# neural network 모델을 구현하는데 __init에서 모델에 대한 정의를 하고 
# forward에서 입력 데이터에 대한 학습이 이루어진다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [15]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [18]:
# 1x28x28
X = torch.rand(1, 28, 28, device=device)
print(X.shape)
logits = model(X)

# what is softmax?
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

torch.Size([1, 28, 28])
Predicted class: tensor([2])


In [19]:
#fashion mnist를 이용해서 이미지가 레이어를 통과할 때 어떻게 변화하는 지 알아보자

input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [23]:
flatten = nn.Flatten()
# input_image를 평면화 시켜서 입력값으로 사용할 수 있도록 만든다.
flat_image = flatten(input_image) 
print(flat_image.size())
print(flat_image)


torch.Size([3, 784])
tensor([[0.6385, 0.7765, 0.7495,  ..., 0.4478, 0.3081, 0.1126],
        [0.6906, 0.7797, 0.1573,  ..., 0.4457, 0.1958, 0.0720],
        [0.9357, 0.7072, 0.3188,  ..., 0.1564, 0.5075, 0.1370]])


In [24]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
# 저장된 가중치와 편향을 사용해서 선형변환을 해주는 객체이다.
# 피처의 개수를 20개로 잡았기 때문에 3x20이라는 사이즈로 변환 되었다.
hidden1 = layer1(flat_image)
print(hidden1.size())
print(hidden1)

torch.Size([3, 20])
tensor([[-0.1667, -0.0524,  0.7626,  0.1126,  0.2006, -0.0221, -0.0410, -0.3301,
          0.0290, -0.8152, -0.4083, -0.4579,  0.0465,  0.4922,  0.0309,  0.3209,
         -0.2345,  0.1336,  0.0117, -0.4032],
        [ 0.1984,  0.1958,  0.6763,  0.0253,  0.4605, -0.7288, -0.4754, -0.1024,
         -0.1001, -0.4928, -0.1733, -0.6313,  0.1807,  0.7136,  0.0755,  0.1859,
         -0.3091,  0.4060,  0.3153, -0.1678],
        [-0.0791, -0.0952,  0.5910,  0.4106,  0.2335, -0.5916, -0.3045, -0.0461,
          0.0936, -0.8907, -0.1051, -0.4521, -0.0895,  0.8127, -0.0230,  0.5106,
         -0.0860,  0.4144,  0.5098, -0.1627]], grad_fn=<AddmmBackward0>)


In [26]:
print(f"Before ReLU: {hidden1}\n\n")
# 비선형 활성화(activation)은 모델의 입출력 사이에 복잡한 관계를 만든다. 데이터의 선형변환 이후에 비선형 활성화가 이루어지며 ReLU이외에도 여러 비선형 기법들이 있다.
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[0.0000, 0.0000, 0.7626, 0.1126, 0.2006, 0.0000, 0.0000, 0.0000, 0.0290,
         0.0000, 0.0000, 0.0000, 0.0465, 0.4922, 0.0309, 0.3209, 0.0000, 0.1336,
         0.0117, 0.0000],
        [0.1984, 0.1958, 0.6763, 0.0253, 0.4605, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1807, 0.7136, 0.0755, 0.1859, 0.0000, 0.4060,
         0.3153, 0.0000],
        [0.0000, 0.0000, 0.5910, 0.4106, 0.2335, 0.0000, 0.0000, 0.0000, 0.0936,
         0.0000, 0.0000, 0.0000, 0.0000, 0.8127, 0.0000, 0.5106, 0.0000, 0.4144,
         0.5098, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.7626, 0.1126, 0.2006, 0.0000, 0.0000, 0.0000, 0.0290,
         0.0000, 0.0000, 0.0000, 0.0465, 0.4922, 0.0309, 0.3209, 0.0000, 0.1336,
         0.0117, 0.0000],
        [0.1984, 0.1958, 0.6763, 0.0253, 0.4605, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1807, 0.7136, 0.0755, 0.1859, 0.0000, 0.4060,
         0.3153, 0.0000],
       

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)# 3x20 -> 3x10
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)
print(logits.shape)

tensor([[-0.2482, -0.0295, -0.0104,  0.0332,  0.3924,  0.1259,  0.2441,  0.3928,
         -0.0671, -0.0063],
        [-0.3081, -0.1292, -0.0110, -0.1331,  0.2747,  0.0865,  0.1125,  0.3512,
         -0.1493, -0.0571],
        [-0.2541, -0.0263, -0.0919, -0.0814,  0.3206,  0.1387,  0.0807,  0.3951,
         -0.1345, -0.1109]], grad_fn=<AddmmBackward0>)
torch.Size([3, 10])


In [31]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)#0~1사이의 값을 반환

tensor([[0.0704, 0.0877, 0.0894, 0.0933, 0.1337, 0.1024, 0.1152, 0.1337, 0.0844,
         0.0897],
        [0.0718, 0.0859, 0.0967, 0.0856, 0.1287, 0.1066, 0.1094, 0.1389, 0.0842,
         0.0923],
        [0.0742, 0.0932, 0.0873, 0.0882, 0.1319, 0.1099, 0.1038, 0.1421, 0.0837,
         0.0857]], grad_fn=<SoftmaxBackward0>)


In [32]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0065, -0.0012,  0.0264,  ..., -0.0268,  0.0340, -0.0107],
        [-0.0157,  0.0070, -0.0343,  ..., -0.0049,  0.0001,  0.0207]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0326, -0.0284], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0359,  0.0327,  0.0049,  ..., -0.0127, -0.0309,  0.0433],
        [ 0.0152,  0.0178,  0.0406,  ..., -0.0124, -0.0060, -0.0149]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias 